In [1]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
import base64
from requests import post, get
import json
import csv
from sklearn import preprocessing

In [2]:
client_id = 'e7c7e1861d3342a0a51e039eba246925' ##ganti variabel dengan client_id milik anda
client_secret = '91470fe8a4d446909fb1ffee1952dfc4' ##ganti variabel dengan client_secret milik anda
playlistId = '6xIyqaVE6pGCXfdslfhhRE'

## 37i9dQZF1DXbrUpGvoi3TS - 1(similar sad songs)
## 1dtCMTYzAOzwKXqklxPJNS - 2(old songs, rock, rap)
## 0IN7IWKmIfwlEysGyWUuRg - 3(mix of modern electronic, pop, and rock)

dataset = []
dataset2 = []
dataset3 = []

In [3]:
def getToken():
    # gabungkan client_id dan client_secret
    auth_string = client_id + ':' + client_secret

    # encode ke base64
    auth_b64 = base64.b64encode(auth_string.encode('utf-8'))

    # url untuk mengambil token
    url = 'https://accounts.spotify.com/api/token'

    # header untuk mengambil token - sesuai dengan guide dari spotify
    headers = {
        'Authorization': 'Basic ' + auth_b64.decode('utf-8'),
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    # data untuk mengambil token - sesuai dengan guide dari spotify
    data = {'grant_type': 'client_credentials'}

    # kirim request POST ke spotify
    result = post(url, headers=headers, data=data)

    # parse response ke json
    json_result = json.loads(result.content)
    token = json_result['access_token']

    # ambil token untuk akses API
    return token

## panggil fungsi getToken() dibawah ini

In [4]:
## pengambilan token untuk otorisasi API
def getAuthHeader(token):
    return {'Authorization': 'Bearer ' + token}

In [5]:
## pengambilan audio features dari track (lagu)
def getAudioFeatures(token, trackId):
    # endpoint untuk akses playlist
    url = f'https://api.spotify.com/v1/audio-features/{trackId}'
    # ambil token untuk otorisasi, gunakan sebagai header
    headers = getAuthHeader(token)
    result = get(url, headers=headers)  # kirim request GET ke spotify
    json_result = json.loads(result.content)  # parse response ke json

    # ambil data yang diperlukan dari response
    audio_features_temp = [
        json_result['danceability'],
        json_result['energy'],
        json_result['key'],
        json_result['loudness'],
        json_result['mode'],
        json_result['speechiness'],
        json_result['acousticness'],
        json_result['instrumentalness'],
        json_result['liveness'],
        json_result['valence'],
        json_result['tempo'],
    ]
    dataset2.append(audio_features_temp)

In [6]:
# pengambilan track (lagu) dari playlist
def getPlaylistItems(token, playlistId):
    # endpoint untuk akses playlist
    url = f'https://api.spotify.com/v1/playlists/{playlistId}/tracks'
    limit = '&limit=100'  # batas maksimal track yang diambil
    market = '?market=ID'  # negara yang tempat aplikasi diakses
    # format data dari track yang diambil
    fields = '&fields=items%28track%28id%2Cname%2Cartists%2Cpopularity%2C+duration_ms%2C+album%28release_date%29%29%29'
    url = url+market+fields+limit  # gabungkan semua parameter
    # ambil token untuk otorisasi, gunakan sebagai header
    headers = getAuthHeader(token)
    result = get(url, headers=headers)  # kirim request GET ke spotify
    json_result = json.loads(result.content)  # parse response ke json
    # print(json_result)

    # ambil data yang diperlukan dari response
    for i in range(len(json_result['items'])):
        playlist_items_temp = []
        playlist_items_temp.append(json_result['items'][i]['track']['id'])
        playlist_items_temp.append(
            json_result['items'][i]['track']['name'].encode('utf-8'))
        playlist_items_temp.append(
            json_result['items'][i]['track']['artists'][0]['name'].encode('utf-8'))
        playlist_items_temp.append(
            json_result['items'][i]['track']['popularity'])
        playlist_items_temp.append(
            json_result['items'][i]['track']['duration_ms'])
        playlist_items_temp.append(
            int(json_result['items'][i]['track']['album']['release_date'][0:4]))
        dataset.append(playlist_items_temp)

    # ambil audio features dari semua track di dalam playlist
    for i in range(len(dataset)):
        getAudioFeatures(token, dataset[i][0])

    # gabungkan dataset dan dataset2
    for i in range(len(dataset)):
        dataset3.append(dataset[i]+dataset2[i])

    print(dataset3)
    # convert dataset3 into csv
    with open('dataset.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["id", "name", "artist", "popularity", "duration_ms", "year", "danceability", "energy", "key", "loudness", "mode",
                         "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"])
        writer.writerows(dataset3)

token = getToken()
print('access token : '+token)
getPlaylistItems(token, playlistId)

access token : BQDgrR74PSJVr9N3qqogqwS6_o53CXNqPRBvx9-sMyVmRkWAXlzCD6w7KOs4oc0hI5eRysLXHLCHdWmI_bHBNPSufDvJXAIQwO3BTJuZNQlHQjO_Cm0
[['1RtlbxsPpDBsHHmGTb7ah2', b'Odd Eye', b'Dreamcatcher', 0, 184413, 2021, 0.537, 0.932, 6, -1.84, 0, 0.0869, 0.0487, 0, 0.197, 0.556, 160.058], ['4uuRFwokX9R4hnOWbreDga', b"Don't Light My Fire", b'Dreamcatcher', 32, 209280, 2021, 0.498, 0.862, 1, -3.966, 1, 0.0375, 0.000221, 0, 0.104, 0.329, 130.035], ['5F5W7RRODrX41Np6M2cK5i', b'Eclipse', b'Dreamcatcher', 35, 203613, 2021, 0.591, 0.962, 1, -2.575, 0, 0.0864, 0.096, 0, 0.12, 0.706, 97.493], ['2u7p3Bzkv8h0Yx7w7R370n', b'BOCA', b'Dreamcatcher', 0, 189546, 2020, 0.684, 0.948, 8, -1.713, 0, 0.0576, 0.00676, 4.7e-05, 0.125, 0.643, 99.981], ['2GOVhdRm7EqSk8k1STWv2x', b'\xeb\xb0\x94\xeb\x9e\x8c\xec\x95\x84 (Wind Blows)', b'Dreamcatcher', 0, 197493, 2021, 0.514, 0.963, 4, -2.931, 0, 0.114, 0.00234, 0, 0.526, 0.591, 164.987], ['6mIjJONoUMvGPT9Kzrab3L', b'LA DI DA', b'EVERGLOW', 58, 210893, 2020, 0.477, 0.85, 3, -1.4

In [7]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import plotly.express as px 

## muat dataset
data = pd.read_csv('dataset.csv')
data.head()

,id,name,artist,popularity,duration_ms,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1RtlbxsPpDBsHHmGTb7ah2,b'Odd Eye',b'Dreamcatcher',0,184413,2021,0.537,0.932,6,-1.840,0,0.0869,0.048700,0.000000,0.197,0.556,160.058
1,4uuRFwokX9R4hnOWbreDga,"b""Don't Light My Fire""",b'Dreamcatcher',32,209280,2021,0.498,0.862,1,-3.966,1,0.0375,0.000221,0.000000,0.104,0.329,130.035
2,5F5W7RRODrX41Np6M2cK5i,b'Eclipse',b'Dreamcatcher',35,203613,2021,0.591,0.962,1,-2.575,0,0.0864,0.096000,0.000000,0.120,0.706,97.493
3,2u7p3Bzkv8h0Yx7w7R370n,b'BOCA',b'Dreamcatcher',0,189546,2020,0.684,0.948,8,-1.713,0,0.0576,0.006760,0.000047,0.125,0.643,99.981
4,2GOVhdRm7EqSk8k1STWv2x,b'\xeb\xb0\x94\xeb\x9e\x8c\xec\x95\x84 (Wind B...,b'Dreamcatcher',0,197493,2021,0.514,0.963,4,-2.931,0,0.1140,0.002340,0.000000,0.526,0.591,164.987


In [8]:
## Hapus karakter yang tidak perlu pada kolom artist dan name
data['artist'] = data['artist'].map(lambda x: str(x)[2:-1])
data['name'] = data['name'].map(lambda x: str(x)[2:-1])

data.head()

,id,name,artist,popularity,duration_ms,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1RtlbxsPpDBsHHmGTb7ah2,Odd Eye,Dreamcatcher,0,184413,2021,0.537,0.932,6,-1.840,0,0.0869,0.048700,0.000000,0.197,0.556,160.058
1,4uuRFwokX9R4hnOWbreDga,Don't Light My Fire,Dreamcatcher,32,209280,2021,0.498,0.862,1,-3.966,1,0.0375,0.000221,0.000000,0.104,0.329,130.035
2,5F5W7RRODrX41Np6M2cK5i,Eclipse,Dreamcatcher,35,203613,2021,0.591,0.962,1,-2.575,0,0.0864,0.096000,0.000000,0.120,0.706,97.493
3,2u7p3Bzkv8h0Yx7w7R370n,BOCA,Dreamcatcher,0,189546,2020,0.684,0.948,8,-1.713,0,0.0576,0.006760,0.000047,0.125,0.643,99.981
4,2GOVhdRm7EqSk8k1STWv2x,\xeb\xb0\x94\xeb\x9e\x8c\xec\x95\x84 (Wind Blows),Dreamcatcher,0,197493,2021,0.514,0.963,4,-2.931,0,0.1140,0.002340,0.000000,0.526,0.591,164.987


In [9]:
##delete empty string in name column
data = data[data['name'] != '']

##reset index
data = data.reset_index(drop=True)
data.head()

,id,name,artist,popularity,duration_ms,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1RtlbxsPpDBsHHmGTb7ah2,Odd Eye,Dreamcatcher,0,184413,2021,0.537,0.932,6,-1.840,0,0.0869,0.048700,0.000000,0.197,0.556,160.058
1,4uuRFwokX9R4hnOWbreDga,Don't Light My Fire,Dreamcatcher,32,209280,2021,0.498,0.862,1,-3.966,1,0.0375,0.000221,0.000000,0.104,0.329,130.035
2,5F5W7RRODrX41Np6M2cK5i,Eclipse,Dreamcatcher,35,203613,2021,0.591,0.962,1,-2.575,0,0.0864,0.096000,0.000000,0.120,0.706,97.493
3,2u7p3Bzkv8h0Yx7w7R370n,BOCA,Dreamcatcher,0,189546,2020,0.684,0.948,8,-1.713,0,0.0576,0.006760,0.000047,0.125,0.643,99.981
4,2GOVhdRm7EqSk8k1STWv2x,\xeb\xb0\x94\xeb\x9e\x8c\xec\x95\x84 (Wind Blows),Dreamcatcher,0,197493,2021,0.514,0.963,4,-2.931,0,0.1140,0.002340,0.000000,0.526,0.591,164.987


In [10]:
##delete empty string in name column
data = data[data['name'] != '']

##reset index
data = data.reset_index(drop=True)
data.head()

,id,name,artist,popularity,duration_ms,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1RtlbxsPpDBsHHmGTb7ah2,Odd Eye,Dreamcatcher,0,184413,2021,0.537,0.932,6,-1.840,0,0.0869,0.048700,0.000000,0.197,0.556,160.058
1,4uuRFwokX9R4hnOWbreDga,Don't Light My Fire,Dreamcatcher,32,209280,2021,0.498,0.862,1,-3.966,1,0.0375,0.000221,0.000000,0.104,0.329,130.035
2,5F5W7RRODrX41Np6M2cK5i,Eclipse,Dreamcatcher,35,203613,2021,0.591,0.962,1,-2.575,0,0.0864,0.096000,0.000000,0.120,0.706,97.493
3,2u7p3Bzkv8h0Yx7w7R370n,BOCA,Dreamcatcher,0,189546,2020,0.684,0.948,8,-1.713,0,0.0576,0.006760,0.000047,0.125,0.643,99.981
4,2GOVhdRm7EqSk8k1STWv2x,\xeb\xb0\x94\xeb\x9e\x8c\xec\x95\x84 (Wind Blows),Dreamcatcher,0,197493,2021,0.514,0.963,4,-2.931,0,0.1140,0.002340,0.000000,0.526,0.591,164.987


In [11]:
## drop name artist and year column
data2 = data.copy()
data2 = data2.drop(['artist', 'name', 'year', 'popularity', 'key','duration_ms', 'mode', 'id'], axis=1)

data2.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.537,0.932,-1.840,0.0869,0.048700,0.000000,0.197,0.556,160.058
1,0.498,0.862,-3.966,0.0375,0.000221,0.000000,0.104,0.329,130.035
2,0.591,0.962,-2.575,0.0864,0.096000,0.000000,0.120,0.706,97.493
3,0.684,0.948,-1.713,0.0576,0.006760,0.000047,0.125,0.643,99.981
4,0.514,0.963,-2.931,0.1140,0.002340,0.000000,0.526,0.591,164.987


In [12]:
from sklearn import preprocessing

## normalize all data to 0 and 1
x = data2.values ##returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data2 = pd.DataFrame(x_scaled)

## convert to dataframe
data2.columns = ['acousticness','danceability','energy','instrumentalness','loudness', 'liveness', 'speechiness', 'tempo','valence']

## data2.head()
data2.describe()

,acousticness,danceability,energy,instrumentalness,loudness,liveness,speechiness,tempo,valence
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.512080,0.751374,0.655867,0.168272,0.132609,0.028731,0.249843,0.554519,0.453097
std,0.196331,0.174466,0.203852,0.180808,0.173450,0.119348,0.217102,0.222820,0.218242
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.388359,0.658175,0.548360,0.053389,0.015261,0.000000,0.089339,0.415087,0.267983
50%,0.527672,0.770142,0.673691,0.117580,0.061870,0.000000,0.175167,0.547376,0.421718
75%,0.651240,0.885071,0.809734,0.211394,0.189643,0.002465,0.352035,0.717566,0.632701
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
## Lakukan PCA untuk mengurangi jumlah fitur menjadi 2 fitur saja
pca = PCA(n_components=2)
pca.fit(data2)
pca_data = pca.transform(data2)

## buat dataframe dari hasil pca
pca_df = pd.DataFrame(data=pca_data, columns=['x', 'y'])
pca_df.head()

## plot pca_df using plotly
fig = px.scatter(pca_df, x='x', y='y', title='PCA')
fig.show()

In [14]:
## rubah bentuk data ke list 
data2 = list(zip(pca_df['x'], pca_df['y']))

## fit kmeans model
kmeans = KMeans(n_init=10, max_iter=1000).fit(data2)

## make scatter plot using plotly
fig = px.scatter(pca_df, x='x', y='y', color=kmeans.labels_, color_continuous_scale='rainbow', hover_data=[data.artist, data.name])
fig.show()